## Predicting 2023 MLB Season: 06 - Merge Pitcher Game Level Data With Model Data
Now that I have raw game-level data for each pitcher, I derive features based on the starting pitchers to help our prediction model for individual games.

In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns',1000)
pd.set_option('display.max_rows',1000)

## Overall Plan of Attack
- For each starting pitcher I will load their raw data, create features for each game based on their previous performance, and then save the dataframe in a dictionary structure for easy lookup
- Then I can iterate through the game-level dataframe, add in the features for each starting pitcher, and use those to improve the model


In [2]:
df_cole = pd.read_csv('./hidden_data/game_level_pitching_logs/pitching_data_coleg001.csv')

In [3]:
df_cole.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 31 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   at_vs        300 non-null    object 
 1   Opponent     300 non-null    object 
 2   League       300 non-null    object 
 3   GS           300 non-null    int64  
 4   CG           300 non-null    int64  
 5   SHO          300 non-null    int64  
 6   GF           300 non-null    int64  
 7   SV           300 non-null    int64  
 8   IP           300 non-null    float64
 9   H            300 non-null    int64  
 10  BFP          300 non-null    int64  
 11  HR           300 non-null    int64  
 12  R            300 non-null    int64  
 13  ER           300 non-null    int64  
 14  BB           300 non-null    int64  
 15  IB           300 non-null    int64  
 16  SO           300 non-null    int64  
 17  SH           300 non-null    int64  
 18  SF           300 non-null    int64  
 19  WP      

In [5]:
df_cole.head(20)

,at_vs,Opponent,League,GS,CG,SHO,GF,SV,IP,H,BFP,HR,R,ER,BB,IB,SO,SH,SF,WP,HBP,BK,2B,3B,GDP,ROE,W,L,ERA,Date,dblhead_num
0,VS,SF,N,1,0,0,0,0,6.1,7,27,0,2,2,0,0,2,1,0,0,1,0,1,0,0,0,1,0,2.84,6-11-2013,NaN
1,VS,LA,N,1,0,0,0,0,5.2,7,23,0,3,3,0,0,1,0,0,0,0,0,1,0,1,0,1,0,3.75,6-16-2013,NaN
2,AT,ANA,A,1,0,0,0,0,6.1,4,24,1,2,2,1,0,5,0,0,0,0,0,0,0,0,0,1,0,3.44,6-21-2013,NaN
3,VS,MIL,N,1,0,0,0,0,6.0,8,27,0,3,3,3,0,3,0,0,0,1,0,1,0,2,0,1,0,3.70,6-28-2013,NaN
4,VS,PHI,N,1,0,0,0,0,5.1,8,23,0,3,3,1,0,5,1,0,0,0,0,1,0,0,0,0,1,3.94,7- 4-2013,NaN
5,VS,OAK,A,1,0,0,0,0,7.0,5,26,1,2,2,2,0,4,0,0,0,0,0,1,0,2,0,0,1,3.68,7- 9-2013,NaN
6,VS,NY,N,1,0,0,0,0,5.0,6,22,0,3,3,2,0,5,0,0,0,0,0,1,0,1,0,0,1,3.89,7-14-2013,NaN
7,AT,WAS,N,1,0,0,0,0,7.0,2,24,1,1,1,1,0,4,0,0,0,1,0,0,0,0,0,1,0,3.51,7-23-2013,NaN
8,AT,MIA,N,1,0,0,0,0,7.0,4,27,1,3,3,2,0,8,0,1,0,0,0,1,0,0,0,0,1,3.56,7-28-2013,NaN
9,VS,COL,N,1,0,0,0,0,5.1,3,21,1,3,3,2,0,6,0,0,1,0,0,0,0,0,0,0,1,3.69,8- 2-2013,NaN


In [6]:
# Turn IP into real decimals opposed to .1,.2,.3
df_cole['IP_real'] = (df_cole.IP - (df_cole.IP % 1)) + (df_cole.IP % 1) * (10/3)
df_cole.head(10)

,at_vs,Opponent,League,GS,CG,SHO,GF,SV,IP,H,BFP,HR,R,ER,BB,IB,SO,SH,SF,WP,HBP,BK,2B,3B,GDP,ROE,W,L,ERA,Date,dblhead_num,IP_real
0,VS,SF,N,1,0,0,0,0,6.1,7,27,0,2,2,0,0,2,1,0,0,1,0,1,0,0,0,1,0,2.84,6-11-2013,NaN,6.333333
1,VS,LA,N,1,0,0,0,0,5.2,7,23,0,3,3,0,0,1,0,0,0,0,0,1,0,1,0,1,0,3.75,6-16-2013,NaN,5.666667
2,AT,ANA,A,1,0,0,0,0,6.1,4,24,1,2,2,1,0,5,0,0,0,0,0,0,0,0,0,1,0,3.44,6-21-2013,NaN,6.333333
3,VS,MIL,N,1,0,0,0,0,6.0,8,27,0,3,3,3,0,3,0,0,0,1,0,1,0,2,0,1,0,3.70,6-28-2013,NaN,6.000000
4,VS,PHI,N,1,0,0,0,0,5.1,8,23,0,3,3,1,0,5,1,0,0,0,0,1,0,0,0,0,1,3.94,7- 4-2013,NaN,5.333333
5,VS,OAK,A,1,0,0,0,0,7.0,5,26,1,2,2,2,0,4,0,0,0,0,0,1,0,2,0,0,1,3.68,7- 9-2013,NaN,7.000000
6,VS,NY,N,1,0,0,0,0,5.0,6,22,0,3,3,2,0,5,0,0,0,0,0,1,0,1,0,0,1,3.89,7-14-2013,NaN,5.000000
7,AT,WAS,N,1,0,0,0,0,7.0,2,24,1,1,1,1,0,4,0,0,0,1,0,0,0,0,0,1,0,3.51,7-23-2013,NaN,7.000000
8,AT,MIA,N,1,0,0,0,0,7.0,4,27,1,3,3,2,0,8,0,1,0,0,0,1,0,0,0,0,1,3.56,7-28-2013,NaN,7.000000
9,VS,COL,N,1,0,0,0,0,5.1,3,21,1,3,3,2,0,6,0,0,1,0,0,0,0,0,0,0,1,3.69,8- 2-2013,NaN,5.333333


In [7]:
# Rolling sum function that will be used on all the metrics
def roll_column(df, col, winsize):
    # do the standard Pandas rolling calc
    t_col = df[col].rolling(winsize, closed='left').sum().to_numpy()
    
    # for the early columns, just do a rolling sum from the beginning
    t_col[:winsize] = np.concatenate(([0],df[col].iloc[:(winsize)].cumsum().to_numpy()[:-1]))

    return(t_col)

In [10]:
def load_and_process_pitch_df(p_id, filepath=''):
    fname = filepath+'pitching_data_'+p_id+'.csv'
    pitch_df = pd.read_csv(fname)
    
    # Convert date, fix dblhead_num to be 0,1,2
    pitch_df['Date'] = (pd.to_datetime(pitch_df.Date).astype(str).str.replace('-','')).astype(int)
    pitch_df.dblhead_num.fillna(0, inplace=True)
    pitch_df['dblhead_num'] = pitch_df['dblhead_num'].astype(int)
    
    # Convert IP to proper mathematical format
    pitch_df['IP_real'] = (pitch_df.IP - (pitch_df.IP % 1)) + (pitch_df.IP % 1) * (10/3)
    
    cols_to_agg = ['IP_real', 'H','BFP', 'HR', 'R', 'ER', 'BB', 'IB', 'SO', 'SH', 'SF', 'WP', 'HBP', 'BK',
       '2B', '3B']
    # Want to look at 10, 25, and 75 games back
    # NOTE: Could a 2-3 game rollsum make an impact on the model?
    winsizes = [10,35,75]
    for winsize in winsizes:
        for raw_col in cols_to_agg:
            new_colname = f'rollsum_{raw_col}_{winsize}'        
            pitch_df[new_colname] = roll_column(pitch_df, raw_col, winsize)

    # Enter default values for new pitchers
    er_per_ip_def = (5/9)
    h_bb_per_ip_def = 1.5
    h_bb_per_bf_def = .37
    so_per_bf_def = .2
    ip_per_game_def = 3
    bf_per_game_def = 12
    tb_bb_perc_def = .45
    fip_numer_per_ip_def = .124*13 + 1.5*3 - 2*.8
    fip_numer_per_bf_def = .03*13 + .37*3 - 2*.2
    # Create rollsums for all metrics
    for winsize in winsizes:
        hit_col = f'rollsum_H_{winsize}'
        bb_col = f'rollsum_BB_{winsize}'
        h_bb_col = f'H_BB_roll_{winsize}'
        double_col = f'rollsum_2B_{winsize}'
        triple_col = f'rollsum_3B_{winsize}'
        hr_col = f'rollsum_HR_{winsize}'
        xb_col = f'XB_roll_{winsize}'
        tb_col = f'TB_roll_{winsize}'
        so_col = f'rollsum_SO_{winsize}'
        so_mod_col = f'SO_mod_{winsize}'
        ip_col = f'rollsum_IP_real_{winsize}'
        ip_mod_col = f'IP_mod_{winsize}'
        er_col = f'rollsum_ER_{winsize}'
        er_mod_col = f'ER_mod_{winsize}'
        bf_col = f'rollsum_BFP_{winsize}'
        bf_mod_col = f'BF_mod_{winsize}'
        era_col = f'ERA_{winsize}'
        fip_col = f'FIP_{winsize}'
        fip_perc_col = f'FIP_perc_{winsize}'
        fip_numer_col = f'FIP_numer_{winsize}'
        fip_numer_mod_col = f'FIP_numer_mod_{winsize}'
        fip_numer_mod2_col = f'FIP_numer_mod2_{winsize}'
        whip_col = f'WHIP_{winsize}'
        so_perc_col = f'SO_perc_{winsize}'
        h_bb_perc_col = f'H_BB_perc_{winsize}'
        h_bb_mod_col = f'H_BB_mod_{winsize}'
        h_bb_mod2_col = f'H_BB_mod2_{winsize}'
        tb_bb_mod_col = f'TB_BB_mod_{winsize}'
        tb_bb_perc_col = f'TB_BB_perc_{winsize}'
        pitch_df[h_bb_col] = pitch_df[hit_col]+pitch_df[bb_col]
        pitch_df[xb_col] = pitch_df[double_col]+2*pitch_df[triple_col]+3*pitch_df[hr_col]
        pitch_df[tb_col] = pitch_df[hit_col]+pitch_df[xb_col]
        pitch_df[ip_mod_col] = np.maximum(pitch_df[ip_col], winsize*ip_per_game_def)
        pitch_df[bf_mod_col] = np.maximum(pitch_df[bf_col], winsize*bf_per_game_def)
        pitch_df[er_mod_col] = pitch_df[er_col] + er_per_ip_def*(pitch_df[ip_mod_col]-pitch_df[ip_col])
        pitch_df[fip_numer_col] = 13*pitch_df[hr_col] + 3*pitch_df[h_bb_col] -2*pitch_df[so_col]
        pitch_df[fip_numer_mod_col] = pitch_df[fip_numer_col] + fip_numer_per_ip_def*(pitch_df[ip_mod_col]-pitch_df[ip_col])
        pitch_df[fip_numer_mod2_col] = pitch_df[fip_numer_col] + fip_numer_per_bf_def*(pitch_df[bf_mod_col]-pitch_df[bf_col])
        pitch_df[h_bb_mod_col] = pitch_df[h_bb_col] + h_bb_per_ip_def*(pitch_df[ip_mod_col]-pitch_df[ip_col])
        pitch_df[h_bb_mod2_col] = pitch_df[h_bb_col] + h_bb_per_bf_def*(pitch_df[bf_mod_col]-pitch_df[bf_col])
        pitch_df[so_mod_col] = pitch_df[so_col] + so_per_bf_def*(pitch_df[bf_mod_col]-pitch_df[bf_col])
        pitch_df[tb_bb_mod_col] = (pitch_df[tb_col] + pitch_df[bb_col])+ tb_bb_perc_def*(pitch_df[bf_mod_col]-pitch_df[bf_col])
        pitch_df[era_col] = (pitch_df[er_mod_col]/pitch_df[ip_mod_col])*9
        pitch_df[fip_col] = (pitch_df[fip_numer_mod_col]/pitch_df[ip_mod_col])
        pitch_df[fip_perc_col] = (pitch_df[fip_numer_mod_col]/pitch_df[bf_mod_col])
        pitch_df[whip_col] = pitch_df[h_bb_mod_col]/pitch_df[ip_mod_col]
        pitch_df[so_perc_col] = pitch_df[so_mod_col]/pitch_df[bf_mod_col]
        pitch_df[tb_bb_perc_col] = pitch_df[tb_bb_mod_col]/pitch_df[bf_mod_col]
        pitch_df[h_bb_perc_col] = pitch_df[h_bb_mod2_col]/pitch_df[bf_mod_col]
    pitch_df['date_dblhead'] = (pitch_df['Date'].astype(str) + pitch_df['dblhead_num'].astype(str)).astype(int)
    pitch_df.set_index('date_dblhead', inplace=True)
    return(pitch_df)

In [11]:
df_cole = load_and_process_pitch_df('coleg001','./hidden_data/game_level_pitching_logs/')

C:\Users\Dylan McGee\AppData\Local\Temp\ipykernel_15348\2760201552.py:68: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  pitch_df[fip_numer_col] = 13*pitch_df[hr_col] + 3*pitch_df[h_bb_col] -2*pitch_df[so_col]
C:\Users\Dylan McGee\AppData\Local\Temp\ipykernel_15348\2760201552.py:69: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  pitch_df[fip_numer_mod_col] = pitch_df[fip_numer_col] + fip_numer_per_ip_def*(pitch_df[ip_mod_col]-pitch_df[ip_col])
C:\Users\Dylan McGee\AppData\Local\Temp\ipykernel_15348\2760201552.py:70: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of c

In [12]:
df_cole.head(50)

,at_vs,Opponent,League,GS,CG,SHO,GF,SV,IP,H,BFP,HR,R,ER,BB,IB,SO,SH,SF,WP,HBP,BK,2B,3B,GDP,ROE,W,L,ERA,Date,dblhead_num,IP_real,rollsum_IP_real_10,rollsum_H_10,rollsum_BFP_10,rollsum_HR_10,rollsum_R_10,rollsum_ER_10,rollsum_BB_10,rollsum_IB_10,rollsum_SO_10,rollsum_SH_10,rollsum_SF_10,rollsum_WP_10,rollsum_HBP_10,rollsum_BK_10,rollsum_2B_10,rollsum_3B_10,rollsum_IP_real_35,rollsum_H_35,rollsum_BFP_35,rollsum_HR_35,rollsum_R_35,rollsum_ER_35,rollsum_BB_35,rollsum_IB_35,rollsum_SO_35,rollsum_SH_35,rollsum_SF_35,rollsum_WP_35,rollsum_HBP_35,rollsum_BK_35,rollsum_2B_35,rollsum_3B_35,rollsum_IP_real_75,rollsum_H_75,rollsum_BFP_75,rollsum_HR_75,rollsum_R_75,rollsum_ER_75,rollsum_BB_75,rollsum_IB_75,rollsum_SO_75,rollsum_SH_75,rollsum_SF_75,rollsum_WP_75,rollsum_HBP_75,rollsum_BK_75,rollsum_2B_75,rollsum_3B_75,H_BB_roll_10,XB_roll_10,TB_roll_10,IP_mod_10,BF_mod_10,ER_mod_10,FIP_numer_10,FIP_numer_mod_10,FIP_numer_mod2_10,H_BB_mod_10,H_BB_mod2_10,SO_mod_10,TB_BB_mod_10,ERA_10,FIP_10,FIP_perc_10,WHIP_10,SO_perc_10,TB_BB_perc_10,H_BB_perc_10,H_BB_roll_35,XB_roll_35,TB_roll_35,IP_mod_35,BF_mod_35,ER_mod_35,FIP_numer_35,FIP_numer_mod_35,FIP_numer_mod2_35,H_BB_mod_35,H_BB_mod2_35,SO_mod_35,TB_BB_mod_35,ERA_35,FIP_35,FIP_perc_35,WHIP_35,SO_perc_35,TB_BB_perc_35,H_BB_perc_35,H_BB_roll_75,XB_roll_75,TB_roll_75,IP_mod_75,BF_mod_75,ER_mod_75,FIP_numer_75,FIP_numer_mod_75,FIP_numer_mod2_75,H_BB_mod_75,H_BB_mod2_75,SO_mod_75,TB_BB_mod_75,ERA_75,FIP_75,FIP_perc_75,WHIP_75,SO_perc_75,TB_BB_perc_75,H_BB_perc_75
date_dblhead,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
201306110,VS,SF,N,1,0,0,0,0,6.1,7,27,0,2,2,0,0,2,1,0,0,1,0,1,0,0,0,1,0,2.84,20130611,0,6.333333,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.000000,120.0,16.666667,0.0,135.360,132.0,45.0,44.40,24.0,54.00,5.000000,4.512000,1.128000,1.500000,0.200000,0.450000,0.370000,0.0,0.0,0.0,105.000000,420.0,58.333333,0.0,473.760,462.0,157.5,155.40,84.0,189.00,5.000000,4.512000,1.128000,1.500000,0.200000,0.450000,0.370000,0.0,0.0,0.0,225.000000,900.0,125.000000,0.0,1015.200,990.0,337.5,333.00,180.0,405.00,5.000000,4.512000,1.128000,1.500000,0.200000,0.450000,0.370000
201306160,VS,LA,N,1,0,0,0,0,5.2,7,23,0,3,3,0,0,1,0,0,0,0,0,1,0,1,0,1,0,3.75,20130616,0,5.666667,6.333333,7.0,27.0,0.0,2.0,2.0,0.0,0.0,2.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,6.333333,7.0,27.0,0.0,2.0,2.0,0.0,0.0,2.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,6.333333,7.0,27.0,0.0,2.0,2.0,0.0,0.0,2.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,7.0,1.0,8.0,30.000000,120.0,15.148148,17.0,123.784,119.3,42.5,41.41,20.6,49.85,4.544444,4.126133,1.031533,1.416667,0.171667,0.415417,0.345083,7.0,1.0,8.0,105.000000,420.0,56.814815,17.0,462.184,449.3,155.0,152.41,80.6,184.85,4.869841,4.401752,1.100438,1.476190,0.191905,0.440119,0.362881,7.0,1.0,8.0,225.000000,900.0,123.481481,17.0,1003.624,977.3,335.0,330.01,176.6,400.85,4.939259,4.460551,1.115138,1.488889,0.196222,0.445389,0.366678
201306210,AT,ANA,A,1,0,0,0,0,6.1,4,24,1,2,2,1,0,5,0,0,0,0,0,0,0,0,0,1,0,3.44,20130621,0,6.333333,12.000000,14.0,50.0,0.0,5.0,5.0,0.0,0.0,3.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,12.000000,14.0,50.0,0.0,5.0,5.0,0.0,0.0,3.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,12.000000,14.0,50.0,0.0,5.0,5.0,0.0,0.0,3.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,14.0,2.0,16.0,30.000000,120.0,15.000000,36.0,117.216,113.0,41.0,39.90,17.0,47.50,4.500000,3.907200,0.976800,1.366667,0.141667,0.395833,0.332500,14.0,2.0,16.0,105.000000,420.0,56.666667,36.0,455.616,443.0,153.5,150.90,77.0,182.50,4.857143,4.339200,1.084800,1.461905,0.183333,0.434524,0.359286,14.0,2.0,16.0,225.000000,900.0,123.333333,36.0,997.056,971.0,333.5,328.50,173.0,398.50,4.933333,4.431360,1.107840,1.482222,0.192222,0.442778,0.365000
201306280,VS,MIL,N,1,0,0,0,0,6.0,8,27,0,3,3,3,0,3,0,0,0,1,0,1,0,2,0,1,0,3.70,20130628,0,6.000000,1

## Load in our game level data

In [13]:
df=pd.read_csv('./hidden_data/mlb_game_and_odds.csv')

C:\Users\Dylan McGee\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\interactiveshell.py:3508: DtypeWarning: Columns (15,85,87) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [16]:
# Get all the pitchers to calculate rollsums for each
start_pitchers_h = df.pitcher_start_id_h.unique()
start_pitchers_v = df.pitcher_start_id_v.unique()
start_pitchers_all = np.union1d(start_pitchers_h, start_pitchers_v)

# Crate a dictionary for each parsing
pitcher_data_dict = {}
for i, p_id in enumerate(start_pitchers_all):
    if i%100==0:
        print(i)
    pitcher_data_dict[p_id] = load_and_process_pitch_df(p_id,'./hidden_data/game_level_pitching_logs/')

0


C:\Users\Dylan McGee\AppData\Local\Temp\ipykernel_15348\2760201552.py:68: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  pitch_df[fip_numer_col] = 13*pitch_df[hr_col] + 3*pitch_df[h_bb_col] -2*pitch_df[so_col]
C:\Users\Dylan McGee\AppData\Local\Temp\ipykernel_15348\2760201552.py:69: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  pitch_df[fip_numer_mod_col] = pitch_df[fip_numer_col] + fip_numer_per_ip_def*(pitch_df[ip_mod_col]-pitch_df[ip_col])
C:\Users\Dylan McGee\AppData\Local\Temp\ipykernel_15348\2760201552.py:70: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of c

100
200
300
400
500
600
700
800
900
1000


In [17]:
np.array(pitcher_data_dict['coleg001'].columns)

array(['at_vs', 'Opponent', 'League', 'GS', 'CG', 'SHO', 'GF', 'SV', 'IP',
       'H', 'BFP', 'HR', 'R', 'ER', 'BB', 'IB', 'SO', 'SH', 'SF', 'WP',
       'HBP', 'BK', '2B', '3B', 'GDP', 'ROE', 'W', 'L', 'ERA', 'Date',
       'dblhead_num', 'IP_real', 'rollsum_IP_real_10', 'rollsum_H_10',
       'rollsum_BFP_10', 'rollsum_HR_10', 'rollsum_R_10', 'rollsum_ER_10',
       'rollsum_BB_10', 'rollsum_IB_10', 'rollsum_SO_10', 'rollsum_SH_10',
       'rollsum_SF_10', 'rollsum_WP_10', 'rollsum_HBP_10',
       'rollsum_BK_10', 'rollsum_2B_10', 'rollsum_3B_10',
       'rollsum_IP_real_35', 'rollsum_H_35', 'rollsum_BFP_35',
       'rollsum_HR_35', 'rollsum_R_35', 'rollsum_ER_35', 'rollsum_BB_35',
       'rollsum_IB_35', 'rollsum_SO_35', 'rollsum_SH_35', 'rollsum_SF_35',
       'rollsum_WP_35', 'rollsum_HBP_35', 'rollsum_BK_35',
       'rollsum_2B_35', 'rollsum_3B_35', 'rollsum_IP_real_75',
       'rollsum_H_75', 'rollsum_BFP_75', 'rollsum_HR_75', 'rollsum_R_75',
       'rollsum_ER_75', 'rollsum_BB_

In [18]:
# Define columns we want to add
raw_cols_to_add = ['GS',  'IP',
       'H', 'BFP', 'HR', 'R', 'ER', 'BB', 'IB', 'SO', 'SH', 'SF', 'WP',
       'HBP', 'BK', '2B', '3B', 'IP_real', 'rollsum_IP_real_10', 'rollsum_H_10',
       'rollsum_BFP_10', 'rollsum_HR_10', 'rollsum_R_10', 'rollsum_ER_10',
       'rollsum_BB_10', 'rollsum_IB_10', 'rollsum_SO_10', 'rollsum_SH_10',
       'rollsum_SF_10', 'rollsum_WP_10', 'rollsum_HBP_10',
       'rollsum_BK_10', 'rollsum_2B_10', 'rollsum_3B_10',
       'rollsum_IP_real_35', 'rollsum_H_35', 'rollsum_BFP_35',
       'rollsum_HR_35', 'rollsum_R_35', 'rollsum_ER_35', 'rollsum_BB_35',
       'rollsum_IB_35', 'rollsum_SO_35', 'rollsum_SH_35', 'rollsum_SF_35',
       'rollsum_WP_35', 'rollsum_HBP_35', 'rollsum_BK_35',
       'rollsum_2B_35', 'rollsum_3B_35', 'rollsum_IP_real_75',
       'rollsum_H_75', 'rollsum_BFP_75', 'rollsum_HR_75', 'rollsum_R_75',
       'rollsum_ER_75', 'rollsum_BB_75', 'rollsum_IB_75', 'rollsum_SO_75',
       'rollsum_SH_75', 'rollsum_SF_75', 'rollsum_WP_75',
       'rollsum_HBP_75', 'rollsum_BK_75', 'rollsum_2B_75',
       'rollsum_3B_75', 'H_BB_roll_10', 'XB_roll_10', 'TB_roll_10',
       'IP_mod_10', 'BF_mod_10', 'ER_mod_10', 'FIP_numer_10',
       'FIP_numer_mod_10', 'FIP_numer_mod2_10', 'H_BB_mod_10',
       'H_BB_mod2_10', 'SO_mod_10', 'TB_BB_mod_10', 'ERA_10', 'FIP_10',
       'FIP_perc_10', 'WHIP_10', 'SO_perc_10', 'TB_BB_perc_10',
       'H_BB_perc_10', 'H_BB_roll_35', 'XB_roll_35', 'TB_roll_35',
       'IP_mod_35', 'BF_mod_35', 'ER_mod_35', 'FIP_numer_35',
       'FIP_numer_mod_35', 'FIP_numer_mod2_35', 'H_BB_mod_35',
       'H_BB_mod2_35', 'SO_mod_35', 'TB_BB_mod_35', 'ERA_35', 'FIP_35',
       'FIP_perc_35', 'WHIP_35', 'SO_perc_35', 'TB_BB_perc_35',
       'H_BB_perc_35', 'H_BB_roll_75', 'XB_roll_75', 'TB_roll_75',
       'IP_mod_75', 'BF_mod_75', 'ER_mod_75', 'FIP_numer_75',
       'FIP_numer_mod_75', 'FIP_numer_mod2_75', 'H_BB_mod_75',
       'H_BB_mod2_75', 'SO_mod_75', 'TB_BB_mod_75', 'ERA_75', 'FIP_75',
       'FIP_perc_75', 'WHIP_75', 'SO_perc_75', 'TB_BB_perc_75',
       'H_BB_perc_75']
cols_to_add = ['Strt_'+col+suff for col in raw_cols_to_add for suff in ['_h','_v']]

col_add_dict = {col:np.zeros(df.shape[0]) for col in cols_to_add}

In [19]:
# For each row get game idenitifier, home/away pitcher, find the pitchers' rollsum states in their game level data and append 
for i in range(df.shape[0]):
    row = df.iloc[i,:]
    if i%1000==0:
        print(i)
    sp_id_v = row['pitcher_start_id_v']
    sp_id_h = row['pitcher_start_id_h']
    date_dblhead = row['date_dblhead']
    if sp_id_v in pitcher_data_dict.keys():
        # Go to the visiting pitchers df
        curr_df = pitcher_data_dict[sp_id_v]
        # Find the date of tha game and get the rollsum metrics
        if date_dblhead in curr_df.index:
            for col in raw_cols_to_add:
                col_add_dict['Strt_'+col+'_v'][i] = curr_df.loc[date_dblhead,col]
        else:
            print(f'no match for {sp_id_v} date {date_dblhead}')
    if sp_id_h in pitcher_data_dict.keys():
        curr_df = pitcher_data_dict[sp_id_h]
        if date_dblhead in curr_df.index:
            for col in raw_cols_to_add:
                col_add_dict['Strt_'+col+'_h'][i] = curr_df.loc[date_dblhead,col]
        else:
            print(f'no match for {sp_id_h} date {date_dblhead}')

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000


In [20]:
# Add everything to the main df
for col in cols_to_add:
    df[col] = col_add_dict[col]

C:\Users\Dylan McGee\AppData\Local\Temp\ipykernel_15348\1091973616.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col] = col_add_dict[col]


In [21]:
df.sample(5)

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info,season_x,run_diff,home_victory,run_total,date_dblhead,BATAVG_162_h,BATAVG_162_v,OBP_162_h,OBP_162_v,SLG_162_h,SLG_162_v,OPS_162_h,OPS_162_v,SB_162_h,SB_162_v,CS_162_h,CS_162_v,ERR_162_h,ERR_162_v,SO_perc_162_h,SO_perc_162_v,BB_perc_162_h,BB_perc_162_v,BATAVG_30_h,BATAVG_30_v,OBP_30_h,OBP_30_v,SLG_30_h,SLG_30_v,OPS_30_h,OPS_30_v,SB_30_h,SB_30_v,CS_30_h,CS_30_v,ERR_30_h,ERR_30_v,SO_perc_30_h,SO_perc_30_v,BB_perc_30_h,BB_perc_30_v,away_point_spread,away_point_spread_line,away_money_line,home_point_spread,home_point_spread_line,home_money_line,over_under,over_line,under_line,implied_prob_h,implied_prob_v,implied_prob_h_mid,over_under_result,Strt_GS_h,Strt_GS_v,Strt_IP_h,Strt_IP_v,Strt_H_h,Strt_H_v,Strt_BFP_h,Strt_BFP_v,Strt_HR_h,Strt_HR_v,Strt_R_h,Strt_R_v,Strt_ER_h,Strt_ER_v,Strt_BB_h,Strt_BB_v,Strt_IB_h,Strt_IB_v,Strt_SO_h,Strt_SO_v,Strt_SH_h,Strt_SH_v,Strt_SF_h,Strt_SF_v,Strt_WP_h,Strt_WP_v,Strt_HBP_h,Strt_HBP_v,Strt_BK_h,Strt_BK_v,Strt_2B_h,Strt_2B_v,Strt_3B_h,Strt_3B_v,Strt_IP_real_h,Strt_IP_real_v,Strt_rollsum_IP_real_10_h,Strt_rollsum_IP_real_10_v,Strt_rollsum_H_10_h,Strt_rollsum_H_10_v,Strt_rollsum_BFP_10_h,Strt_rollsum_BFP_10_v,Strt_rollsum_HR_10_h,Strt_rollsum_HR_10_v,Strt_rollsum_R_10_h,Strt_rollsum_R_10_v,Strt_rollsum_ER_10_h,Strt_rollsum_ER_10_v,Strt_rollsum_BB_10_h,Strt_rollsum_BB_10_v,Strt_rollsum_IB_10_h,Strt_rollsum_IB_10_v,Strt_rollsum_SO_10_h,Strt_rollsum_SO_10_v,Strt_rollsum_SH_10_h,Strt_rollsum_SH_10_v,Strt_rollsum_SF_10_h,Strt_rollsum_SF_10_v,Strt_rollsum_WP_10_h,Strt_rollsum_WP_10_v,Strt_rollsum_HBP_10_h,Strt_rollsum_HBP_10_v,Strt_rollsum_BK_10_h,Strt_rollsum_BK_10_v,Strt_rollsum_2B_10_h,Strt_rollsum_2B_10_v,Strt_rollsum_3B_10_h,Strt_rollsum_3B_10_v,Strt_rollsum_IP_real_35_h,Strt_rollsum_IP_real_35_v,Strt_rollsum_H_35_h,Strt_rollsum_H_35_v,Strt_rollsum_BFP_35_h,Strt_rollsum_BFP_35_v,Strt_rollsum_HR_35_h,Strt_rollsum_HR_35_v,Strt_rollsum_R_35_h,Strt_rollsum_R_35_v,Strt_rollsum_ER_35_h,Strt_rollsum_ER_35_v,Strt_rollsum_BB_35_h,Strt_rollsum_BB_35_v,Strt_rollsum_IB_35_h,Strt_rollsum_IB_35_v,Strt_rollsum_SO_35_h,Strt_rollsum_SO_35_v,Strt_rollsum_SH_35_h,Strt_rollsum_SH_35_v,Strt_rollsum_SF_35_h,Strt_rollsum_SF_35_v,Strt_rollsum_WP_35_h,Strt_rollsum_WP_35_v,Strt_rollsum_HBP_35_h,Strt_rollsum_HBP_35_v,Strt_rollsum_BK_35_h,Strt_rollsum_BK_35_v,Strt_rollsum_2B_35_h,Strt_roll

In [22]:
df.shape

(22801, 467)

In [28]:
# Check if there are any pitchers that dont have any data
print((df.Strt_IP_mod_10_h==0).sum())
print((df.Strt_IP_mod_10_v==0).sum())

0
0


In [29]:
df.to_csv('./hidden_data/mlb_game_and_odds_06.csv', index=False)